import files from imdb data, filter to desired results. From a brief look at the data I can see that 1, there are movies that haven't come out yet; 2, there are movies without genres. Since neither of these will be helpful for my analysis I have decided to remove them. Additionally my rationale has determined that I don't want to look at movies before 2014 so I've removed those too.

In [1]:
import pandas as pd
df_imdb_basics = pd.read_csv('zippedData/imdb.title.basics.csv.gz', compression='gzip')
df_filtered = df_imdb_basics[(df_imdb_basics['start_year'] <= 2020) & (df_imdb_basics['start_year'] >= 2014)]
df_imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz', compression='gzip')
df_imdb_ratings = df_imdb_ratings[(df_imdb_ratings['numvotes'] > 100)]

dropna values from genre because that will be a factor in analysis

In [2]:
df_filtered = df_filtered.dropna(subset=['genres'])

split genre out into multiple columns

In [3]:
genres = df_filtered['genres'].str.split(",", n = 1, expand = True)
df_filtered['genre1']= genres[0]
df_filtered['genre2']= genres[1]
genres2 = df_filtered['genre2'].str.split(",", n = 1, expand = True)
df_filtered['genre2']= genres2[0]
df_filtered['genre3']= genres2[1]
df_filtered.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre1,genre2,genre3
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Biography,Drama,None
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",Comedy,Drama,None
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",Comedy,Drama,Fantasy
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy,Comedy,None,None


In [4]:
from pandasql import sqldf
pysqldf =  lambda q: sqldf(q, globals())

In [5]:
q = '''SELECT *
        FROM df_filtered
        JOIN df_imdb_ratings
        USING(tconst)
        ;'''

imdb_joined_df = pysqldf(q)
imdb_joined_df.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,genre1,genre2,genre3,averagerating,numvotes
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama,None,None,6.9,4517
1,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",Comedy,Drama,Fantasy,6.5,119
2,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",Adventure,Animation,Comedy,8.1,263
3,tt0315642,Wazir,Wazir,2016,103.0,"Action,Crime,Drama",Action,Crime,Drama,7.1,15378
4,tt0331314,Bunyan and Babe,Bunyan and Babe,2017,84.0,"Adventure,Animation,Comedy",Adventure,Animation,Comedy,5.0,302
